In [2]:
import pandas as pd
from app.utils.subgraph import KlerosBoardSubgraph
from plotly import graph_objects as go


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
kb_eth = KlerosBoardSubgraph(network='mainnet')
kb_gno = KlerosBoardSubgraph(network='gnosis')
eth_color = dict(color='#7F00FF')
gno_color = dict(color='#009BFF')

In [ ]:
stakes_eth = kb_eth.getAllStakeSets()
stakes_gno = kb_gno.getAllStakeSets()

In [ ]:
df_stakes_eth = pd.DataFrame(stakes_eth)
df_stakes_gno = pd.DataFrame(stakes_gno)

In [ ]:
# PNK staked history
from datetime import datetime
from app.utils.utils import getTimeSeriePNKStaked


pnk_staked_eth = getTimeSeriePNKStaked(df_stakes_eth)
pnk_staked_gno = getTimeSeriePNKStaked(df_stakes_gno)



total_supply = 776626704
pnk_staked_eth['total_supply'] = 776626704
pnk_staked_eth.loc[pnk_staked_eth.index < datetime(year=2024,month=1,day=20), 'total_supply'] -= 12000000  # mint kip66
pnk_staked_eth.loc[pnk_staked_eth.index < datetime(year=2019,month=12,day=30), 'total_supply'] -= 200000000  # mint second token sale

pnk_staked_eth['percentage'] = pnk_staked_eth['total_staked'] / pnk_staked_eth['total_supply']
pnk_staked_gno['percentage'] = pnk_staked_gno['total_staked'] / pnk_staked_eth['total_supply']


In [ ]:
df = pd.merge_asof(
    left=pnk_staked_eth, right=pnk_staked_gno,
    left_index=True, right_index=True,
    suffixes=['', '_gno'],
)

# pnk_staked_eth2 = pnk_staked_eth.assign(type='eth')
# pnk_staked_gno2 = pnk_staked_gno.assign(type='gno')
# df_staked = pd.concat([pnk_staked_eth2, pnk_staked_gno2], axis=0).sort_index()
# print(df_staked.to_string())
# print(df.iloc[-7])

# stacked_df = df.stack()
# print(stacked_df)

fig = go.Figure(data=[
    go.Bar(x=df.index, y=df['percentage'] * 100,
           name="Ethereum", offsetgroup='a', marker=eth_color),
    go.Bar(x=df.index, y=df['percentage_gno'] * 100, name="Gnosis",
           offsetgroup='a', base=df['percentage'] * 100, marker=gno_color),
]
)
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False, range=[0, 30]),
    title=dict(text="% PNK staked over Total Supply")
)
fig.show()

In [ ]:
from app.utils.utils import getTimeSerieActiveJurors

active_jurors_eth: pd.DataFrame = getTimeSerieActiveJurors(df_stakes_eth)
active_jurors_gno: pd.DataFrame = getTimeSerieActiveJurors(df_stakes_gno)


In [ ]:
print(f'Active Jurors:\nMainnet:{active_jurors_eth.iloc[-1].values[0]}\nGnosis:{active_jurors_gno.iloc[-1].values[0]}')

In [ ]:


fig = go.Figure(data=[
    go.Scatter(x=active_jurors_eth.index, y=active_jurors_eth['active_jurors'], mode="lines", name="Ethereum", marker=eth_color),
    go.Scatter(x=active_jurors_gno.index, y=active_jurors_gno['active_jurors'], mode="lines", name="Gnosis", marker=gno_color),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Daily Active Jurors")
)
fig.show()


In [ ]:
growth_eth = active_jurors_eth.diff()
growth_gno = active_jurors_gno.diff()

growth_eth = growth_eth.resample('M').sum()
growth_gno = growth_gno.resample('M').sum()

fig = go.Figure(data=[
    go.Bar(x=growth_eth.index, y=growth_eth['active_jurors'], name="Ethereum", marker=dict(color='#7F00FF')),
    go.Bar(x=growth_gno.index, y=growth_gno['active_jurors'], name="Gnosis", marker=dict(color='#009BFF')),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly Active Jurors Growth")
)
fig.add_hline(y=0)
fig.show()


In [ ]:
# Staked amount of PNKs over history


In [ ]:
txs_eth = kb_eth.getAllTransactions()
txs_gno = kb_gno.getAllTransactions()


In [ ]:
daily_txs_eth = txs_eth.resample('M', on='timestamp').count()
daily_txs_gno = txs_gno.resample('M', on='timestamp').count()


fig = go.Figure(data=[
    go.Bar(x=daily_txs_eth.index, y=daily_txs_eth['tx'], name="Ethereum", marker=eth_color),
    go.Bar(x=daily_txs_gno.index, y=daily_txs_gno['tx'], name="Gnosis", marker=gno_color),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly txs in Kleros Courts")
)
fig.add_hline(y=0)
fig.show()

In [ ]:
token_txs_eth = kb_eth.getAllTransfers()
token_txs_gno = kb_gno.getAllTransfers()

In [ ]:
transfers_eth = pd.DataFrame(token_txs_eth)
transfers_gno = pd.DataFrame(token_txs_gno)

transfers_eth['timestamp'] = pd.to_datetime(transfers_eth.timestamp, unit='s')
transfers_gno['timestamp'] = pd.to_datetime(transfers_gno.timestamp, unit='s')
transfers_eth.sort_values('timestamp', inplace=True)
transfers_gno.sort_values('timestamp', inplace=True)


In [ ]:
# Get ETH price
from app.utils.oracles import CoinGecko
from datetime import datetime, timedelta

days_before = (datetime.now() - transfers_eth.timestamp.min()).days
eth_price = CoinGecko().getETHhistoricPrice(days_before)
eth_price = pd.DataFrame(eth_price, columns=['timestamp', 'price'])
eth_price['timestamp'] = pd.to_datetime(eth_price['timestamp'], unit='ms')


In [ ]:

transfers_eth_price = pd.merge_asof(
    left=transfers_eth, right=eth_price,
    on='timestamp', direction='forward', tolerance=timedelta(hours=23)
    )
transfers_eth_price['ETHAmount_usd'] = transfers_eth_price['ETHAmount'] * transfers_eth_price['price']

In [ ]:

transfers_eth_resampled = transfers_eth_price.resample('M', on='timestamp')['ETHAmount_usd'].sum()
transfers_gno_resampled = transfers_gno.resample('M', on='timestamp')['ETHAmount'].sum()

In [ ]:
fig = go.Figure(data=[
    go.Bar(x=transfers_eth_resampled.index, y=transfers_eth_resampled, name="Ethereum", marker=eth_color),
    go.Bar(x=transfers_gno_resampled.index, y=transfers_gno_resampled, name="Gnosis", marker=gno_color),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly Fees payed to jurors [USD]")
)
fig.add_hline(y=0)
fig.show()


fig = go.Figure(data=[
    go.Scatter(x=transfers_eth_resampled.index, y=transfers_eth_resampled.cumsum(), name="Ethereum", mode='lines', marker=eth_color),
    go.Scatter(x=transfers_gno_resampled.index, y=transfers_gno_resampled.cumsum(), name="Gnosis", mode='lines', marker=gno_color),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Cumulative Fees payed to jurors [USD]")
)
fig.add_hline(y=0)
fig.show()

In [ ]:
jurors_eth = pd.DataFrame(kb_eth.getAllJurors())
jurors_gno = pd.DataFrame(kb_gno.getAllJurors())

In [ ]:
jurors_eth = jurors_eth[jurors_eth.ethRewards > 0][['totalStaked', 'id', 'ethRewards', 'numberOfDisputesAsJuror']]
jurors_gno = jurors_gno[jurors_gno.ethRewards > 0][['totalStaked', 'id', 'ethRewards', 'numberOfDisputesAsJuror']]


In [ ]:
from app.utils.utils import gini


gini_eth = gini(jurors_eth['ethRewards'], jurors_eth['totalStaked'])
gini_gno = gini(jurors_gno['ethRewards'], jurors_gno['totalStaked'])

print(f'Gini Ethereum Court: {gini_eth}\nGini Gnosis Court: {gini_gno}')

In [ ]:
from app.utils.utils import getActiveJurors


jurors_eth = getActiveJurors(df_stakes_eth)
jurors_gno = getActiveJurors(df_stakes_gno)
jurors_eth.reset_index(inplace=True)
jurors_gno.reset_index(inplace=True)
jurors = pd.concat([jurors_eth, jurors_gno],axis=0)
jurors = jurors[jurors['newTotalStake'] > 10000]

jurors.drop_duplicates(subset='address', inplace=True)
jurors.reset_index(inplace=True, drop=True)
jurors.to_csv('active_jurors_gt_10k.csv')

In [ ]:
# Disputes opened
all_disputes_eth = kb_eth.getAllDisputes()
all_disputes_gno = kb_gno.getAllDisputes()

df_disputes_eth = pd.DataFrame(all_disputes_eth)
df_disputes_gno = pd.DataFrame(all_disputes_gno)
df_disputes_eth['startTime'] = pd.to_datetime(df_disputes_eth['startTime'], unit='s')
df_disputes_gno['startTime'] = pd.to_datetime(df_disputes_gno['startTime'], unit='s')
df_disputes_eth.sort_values('startTime', ascending=True, inplace=True)
df_disputes_gno.sort_values('startTime', ascending=True, inplace=True)


In [ ]:
disputes_resampled_eth = df_disputes_eth[['id', 'startTime']].resample('M', on='startTime').count()
disputes_resampled_gno = df_disputes_gno[['id', 'startTime']].resample('M', on='startTime').count()

fig = go.Figure(data=[
    go.Bar(x=disputes_resampled_eth.index, y=disputes_resampled_eth.id, name="Ethereum", marker=eth_color),
    go.Bar(x=disputes_resampled_gno.index, y=disputes_resampled_gno.id, name="Gnosis",  marker=gno_color),
    ]
    )
fig.update_layout(
     xaxis=dict(showgrid=False), 
     yaxis=dict(showgrid=False),
     title=dict(text="Monthly Disputes Creation")
)
fig.add_hline(y=0)
fig.show()